In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import os
import gc
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

# =============================================================================
# 1. CONFIGURAÇÕES
# =============================================================================
INPUT_FILE = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\dataset_treino_final.parquet"
OUTPUT_FILE = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\dataset_limpo.parquet"

# Colunas que NÃO ajudam a detectar ataques (apenas identificam quem/quando)
# Removemos para evitar que o modelo "decore" IPs específicos (Overfitting)
COLS_TO_DROP = [
    'Flow ID', 'Flow_ID', 
    'Source IP', 'Source_IP', 
    'Destination IP', 'Destination_IP', 
    'Timestamp', 
    'SimillarHTTP', 
    'Unnamed: 0', 'origem_arquivo'
]

# Mapa para corrigir nomes estranhos de ataques (Encoding issues)
LABEL_FIX = {
    'Web Attack ï¿½ Brute Force': 'Web Attack - Brute Force',
    'Web Attack ï¿½ XSS': 'Web Attack - XSS',
    'Web Attack ï¿½ Sql Injection': 'Web Attack - Sql Injection'
}

# =============================================================================
# 2. PROCESSO DE LIMPEZA (CHUNKING)
# =============================================================================
def limpar_dataset():
    print(">>> INICIANDO HIGIENIZAÇÃO DE DADOS (LOW RAM MODE) <<<")
    
    # Abre o arquivo em modo leitura de metadados
    try:
        parquet_file = pq.ParquetFile(INPUT_FILE)
    except Exception as e:
        print(f"❌ Erro ao abrir arquivo: {e}")
        return

    writer = None
    total_chunks = parquet_file.num_row_groups
    
    for i in tqdm(range(total_chunks), desc="Limpando Blocos"):
        # 1. Lê um pedaço do arquivo
        df_chunk = parquet_file.read_row_group(i).to_pandas()
        
        # 2. Remove colunas identificadoras (IPs, IDs, etc)
        # errors='ignore' porque algumas podem não existir em todos os chunks
        df_chunk.drop(columns=COLS_TO_DROP, inplace=True, errors='ignore')
        
        # 3. Correção de Labels (Caracteres estranhos)
        if 'Label' in df_chunk.columns:
            df_chunk['Label'] = df_chunk['Label'].replace(LABEL_FIX)
            # Remove espaços extras
            df_chunk['Label'] = df_chunk['Label'].str.strip()
        
        # 4. TRATAMENTO DE INFINITOS E NULOS (CRÍTICO!)
        # Seleciona apenas colunas numéricas
        cols_numericas = df_chunk.select_dtypes(include=[np.number]).columns
        
        # Substitui Infinito por NaN (para depois tratar tudo junto)
        df_chunk[cols_numericas] = df_chunk[cols_numericas].replace([np.inf, -np.inf], np.nan)
        
        # Preenche NaN com 0
        # (Em fluxos de rede, geralmente dado ausente/nulo estatístico equivale a zero)
        df_chunk[cols_numericas] = df_chunk[cols_numericas].fillna(0)
        
        # 5. Salva o bloco limpo no novo arquivo
        table = pa.Table.from_pandas(df_chunk)
        
        if writer is None:
            writer = pq.ParquetWriter(OUTPUT_FILE, table.schema, compression='snappy')
        
        writer.write_table(table)
        
        # Limpa memória RAM imediatamente
        del df_chunk, table
        gc.collect()

    if writer:
        writer.close()
    
    print("\n✅ LIMPEZA CONCLUÍDA COM SUCESSO!")
    print(f"💾 Dataset pronto para IA salvo em: {OUTPUT_FILE}")

# =============================================================================
# 3. EXECUÇÃO
# =============================================================================
if __name__ == "__main__":
    # Remove arquivo antigo se existir para não duplicar dados
    if os.path.exists(OUTPUT_FILE):
        try:
            os.remove(OUTPUT_FILE)
        except OSError:
            print("⚠️ Não foi possível remover o arquivo antigo. Reinicie o Kernel se der erro.")
            
    limpar_dataset()

>>> INICIANDO HIGIENIZAÇÃO DE DADOS (LOW RAM MODE) <<<


Limpando Blocos: 100%|███████████████████████████████████████████████████████████████████| 5/5 [00:43<00:00,  8.78s/it]



✅ LIMPEZA CONCLUÍDA COM SUCESSO!
💾 Dataset pronto para IA salvo em: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\dataset_limpo.parquet
